In [33]:
import torch

# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    # padded_images = torch.zeros(input_images.shape[0], input_images.shape[1], input_images.shape[2]+2, input_images.shape[3]+2)
    # for img_idx, img in enumerate(input_images):
    #     for channel in range(img.shape[0]):
    #         padded_images[img_idx][channel] = torch.nn.functional.pad(img[channel], pad=[1,1,1,1])
            
    # padded_images = torch.zeros([2, 3, 5, 5], dtype=torch.float32)
    # padded_images[:, :, 1:-1, 1:-1] += input_images[:, :, :, :].type(torch.FloatTensor)
    
    padded_images = torch.nn.functional.pad(input_images, pad=(1,1,1,1)).float()
    
    return padded_images


correct_padded_images = torch.tensor(
       [[[[0.,  0.,  0.,  0.,  0.],
          [0.,  0.,  1.,  2.,  0.],
          [0.,  3.,  4.,  5.,  0.],
          [0.,  6.,  7.,  8.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0.,  9., 10., 11.,  0.],
          [0., 12., 13., 14.,  0.],
          [0., 15., 16., 17.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 18., 19., 20.,  0.],
          [0., 21., 22., 23.,  0.],
          [0., 24., 25., 26.,  0.],
          [0.,  0.,  0.,  0.,  0.]]],


        [[[0.,  0.,  0.,  0.,  0.],
          [0., 27., 28., 29.,  0.],
          [0., 30., 31., 32.,  0.],
          [0., 33., 34., 35.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 36., 37., 38.,  0.],
          [0., 39., 40., 41.,  0.],
          [0., 42., 43., 44.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 45., 46., 47.,  0.],
          [0., 48., 49., 50.,  0.],
          [0., 51., 52., 53.,  0.],
          [0.,  0.,  0.,  0.,  0.]]]])

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(torch.allclose(get_padding2d(input_images), correct_padded_images))

True


In [47]:
import numpy as np


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    out_shape = (
        input_matrix_shape[0],
        out_channels,
        (input_matrix_shape[2] + padding * 2 - kernel_size) // stride + 1,
        (input_matrix_shape[3] + padding * 2 - kernel_size) // stride + 1
    )
    return out_shape

print(np.array_equal(
    calc_out_shape(input_matrix_shape=[2, 3, 10, 10],
                   out_channels=10,
                   kernel_size=3,
                   stride=1,
                   padding=0),
    [2, 10, 8, 8]))

# ... и ещё несколько подобных кейсов

True


# В следующих примерах реализуем свертку сами
**Обратите внимание, что везде отсутсвует bias!**

In [48]:
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
## Абстрактный слой в данном случае нужен для того, чтобы не перезадавать переменные
## Если мы делаем 2 разных класса, которые делают что-то похожее,
##  то общую часть можно записать здесь и при создании нового класса наследовать этот
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel
## ниже указан декоратор @abstractmethod. Это значит, что при создании 
## нового класса на основании этого функцию нужно будет переопределить
## иначе будет ошибка
    @abstractmethod
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
## Это класс который выполняет свёртку двумерного слоя
## согласно документации https://pytorch.org/docs/stable/nn.html#conv2d
## выполняется это следующим образом. Сам conv2d это класс. 
## Для начала нужно передать ему параметры. После этого вызвать как функцию 
## с матрицей, которую нужно свернуть. 
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)
     


# функция, создающая объект класса cls и возвращающая свертку от input_matrix
## Эта функия подготоавливает данные и вызывает нужный класс для свертки
## Сделано это для упрощения. 
## Обратите внимание самым первым параметром указывается
## класс, которым нужно свернуть матрицу   conv2d_layer_class

def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
## Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
## Функция тестирует conv2d_layer_class. 

def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    print('Вызвана функция test_conv2d_layer ей передан на тестирование класс ',conv2d_layer_class)
    
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])
    print('Kernel с помощю которого будет выполняться сворачивание \n', kernel)
    in_channels = kernel.shape[1]

## создается тензор размерности 6 таблиц 4*4 
## (то есть высота- 4, ширина -4) количество каналов - 3 (берется из размера kernel)
## и 2 батча 2*3=6
    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()).reshape(batch_size, in_channels, input_height, input_width)
    print('Матрица, которую надо будет свернуть \n', input_tensor)
 ## Здесь вызываются 2 фукнции, которые выполняют сворачивание
 ## Одной передается класс conv2d_layer_class другой Conv2d
 ## Обратите внимание что conv2d_layer_class это аргумент текущей функции 
 ## Эту функцию вызвали через print(test_conv2d_layer(Conv2d))
 ## то есть по факту сравнивается Conv2d и Conv2d
 
    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    print('Класс ',conv2d_layer_class,' выдает свернутую матрицу \n',custom_conv2d_out)

    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    print('Класс ',Conv2d,' выдает свернутую матрицу \n',conv2d_out)

    return torch.allclose(custom_conv2d_out, conv2d_out) and (custom_conv2d_out.shape == conv2d_out.shape)

print(test_conv2d_layer(Conv2d))

Вызвана функция test_conv2d_layer ей передан на тестирование класс  <class '__main__.Conv2d'>
Kernel с помощю которого будет выполняться сворачивание 
 tensor([[[[ 0.,  1.,  0.],
          [ 1.,  2.,  1.],
          [ 0.,  1.,  0.]],

         [[ 1.,  2.,  1.],
          [ 0.,  3.,  3.],
          [ 0.,  1., 10.]],

         [[10., 11., 12.],
          [13., 14., 15.],
          [16., 17., 18.]]]])
Матрица, которую надо будет свернуть 
 tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]],


        [[[48., 49., 50., 51.],
          [52., 53., 54., 55.],
          [56., 57., 58., 59.],
          [60., 61., 62., 63.]],

         [[64., 65., 66., 67.],
     

# Делаем конв слой циклами

In [89]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        
        batch_size, out_channels, output_height, output_width = calc_out_shape(
                                input_tensor.shape, 
                                self.out_channels,
                                self.kernel_size,
                                self.stride,
                                padding=0)
            
        # создадим выходной тензор, заполненный нулями         
        output_tensor = torch.zeros(batch_size, out_channels, output_height, output_width)
        
        # вычисление свертки с использованием циклов.
        # цикл по входным батчам(изображениям)
        for num_batch, batch in enumerate(input_tensor): 
             
            # цикл по фильтрам (количество фильтров совпадает с количеством выходных каналов)  
            for num_kernel, kernel in enumerate(self.kernel):
            
                # цикл по размерам выходного изображения
                for i in range(output_height):
                    for j in range(output_width): 
                        
                        # вырезаем кусочек из батча (сразу по всем входным каналам)
                        current_row = self.stride*i
                        current_column = self.stride*j
                        current_slice = batch[:, current_row:current_row + self.kernel_size, current_column:current_column + self.kernel_size]
                        
                        # умножаем кусочек на фильтр
                        res = float((current_slice * kernel).sum())
                        
                        # заполняем ячейку в выходном тензоре
                        output_tensor[num_batch,num_kernel,i,j] = res
                        
        return output_tensor

# Корректность реализации определится в сравнении со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dLoop))

True


# Избавляемся от циклов, вводим матричные операции

In [98]:
class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида.
    
    # Вариант для любой размерности кернелов, числа входных и выходных каналов, размеров изображений. Присутствует два вложенных цикла. 
    # def _unsqueeze_kernel(self, torch_input, output_height, output_width):
    #     _, in_channels, in_height, in_width = torch_input.shape
    #     ku_size = [self.out_channels, output_height, output_width, in_channels, in_height, in_width]
    #     kernel_unsqueezed = torch.zeros(ku_size, dtype=torch.float32)
    #     for i in range(output_height):
    #         for j in range(output_width):
    #             h_slice = slice(i*self.stride, i*self.stride+self.kernel_size)
    #             w_slice = slice(j*self.stride, j*self.stride+self.kernel_size)
    #             kernel_unsqueezed[:, i, j, :, h_slice, w_slice] = self.kernel.type(torch.float32)
    #     return kernel_unsqueezed.view(-1, in_channels*in_height*in_width)
    
    # без циклов, для всяких размерностей. тренировка torch-а и матричных махинаций
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        img_size_rows = torch_input.shape[2]
        img_size_cols = torch_input.shape[3]
        as_oneline_filters = (self.out_channels, 1, -1)

        # шаблон - "одно применение" ядра
        m = torch.nn.ZeroPad2d((0, img_size_cols - self.kernel_size, 
                                0, img_size_rows - self.kernel_size))
        kernel_unsqueezed = m(self.kernel).view(as_oneline_filters)

        # "применение" ядра к строке изображения
        # добавляем шаблону "self.stride" нулей 
        m = torch.nn.ConstantPad1d((0, self.stride), 0)
        kernel_unsqueezed = m(kernel_unsqueezed)
        # и создаем "output_width" копий для каждого фильтра
        copier = torch.ones(self.out_channels, output_width, 1)
        kernel_unsqueezed = (copier @ kernel_unsqueezed)
        # вытягиваем каждый фильтир в строку и обрезаем хвост
        m = torch.nn.ConstantPad1d((0, - self.stride * output_width), 0)
        kernel_unsqueezed = m(kernel_unsqueezed.view(as_oneline_filters))
        
        # аналогично для "применения" ядра по строкам изображения
        # копируем полученный шаблон "output_width" раз со сдвигом
        m = torch.nn.ConstantPad1d((0, self.stride * img_size_cols), 0)
        kernel_unsqueezed = m(kernel_unsqueezed)
        # создаем "output_height" копий для каждого фильтра
        copier = torch.ones(self.out_channels, output_height, 1)
        kernel_unsqueezed = (copier @ kernel_unsqueezed)
        # вытягиваем каждый фильтир в строку и обрезаем хвост
        m = torch.nn.ConstantPad1d((0, - self.stride * output_height * img_size_cols), 0)
        kernel_unsqueezed = m(kernel_unsqueezed.view(as_oneline_filters))

        # вытянутым в строку фильтрам придаем требуемый размер
        return kernel_unsqueezed.view((self.out_channels * output_height * output_width, -1))

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrix))

True


# Ускоряем еще лучше - избавляемся от кучи нулей в матрицах

In [99]:
class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        converted_kernel = self.kernel.flatten(start_dim=1)
        return converted_kernel

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        converted_input = torch.zeros([torch_input.size(0), 
                                       self.in_channels * self.kernel_size**2, 
                                       output_width * output_height])
        ks = self.kernel_size
        sd = self.stride
        for h in range(output_height):
            for w in range(output_width):
                res = torch_input[:, :, h*sd:h*sd+ks, w*sd:w*sd+ks].flatten(start_dim=1)
                converted_input[:, :, h * output_width + w] = res
        return converted_input

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.view(torch_input.shape[0],
                                                     self.out_channels, output_height,
                                                     output_width)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrixV2))

True
